In [1]:
#importing the packages that are needed in order to create natural prompts
import json
import csv
import pandas as pd
import re
from datetime import datetime 
import random
import os
import json

In [2]:
%pip install transformers

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install accelerate

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
prompts = []
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer,)

In [46]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.5,
    "do_sample": True,
}

In [47]:
def decapitalize(s):
    if not s:
        return s
    return s[0].lower() + s[1:]

In [48]:
final_dataset = []

with open("final.json", "r") as f:
    final_dataset = json.loads(f.read())

In [49]:
import random

In [50]:
prompt_templates_type_a = []
with open("prompt_templates_type_a.json", "r") as f:
    prompt_templates_type_a = json.loads(f.read())

In [51]:
prompt_templates_type_b = []
with open("prompt_templates_type_b.json", "r") as f:
    prompt_templates_type_b = json.loads(f.read())

In [52]:
def fill_template_type_a(chart_json: dict):
    template = random.choice(prompt_templates_type_a)
    
    labels_values = []
    for label in chart_json["data"]:
        labels_values.append(f'{label["label"]} ({label["value"]})')
    labels_values = ", ".join(labels_values)

    return template.format(
        chart_title=decapitalize(chart_json["chart_title"]), 
        chart_type=chart_json["chart_type"], 
        labels_values=labels_values
    )

In [53]:
def fill_template_type_b(chart_json: dict):
    template = random.choice(prompt_templates_type_b)
    
    labels = []
    values = []
    for label in chart_json["data"]:
        labels.append(label["label"])
        values.append(str(label["value"]))
        
    labels = ", ".join(labels)
    values = ", ".join(values)

    return template.format(
        chart_title=decapitalize(chart_json["chart_title"]), 
        chart_type=chart_json["chart_type"], 
        labels=labels,
        values=values
    )

In [54]:
def fill_template(chart_json: dict):
    fill_functions = [fill_template_type_a, fill_template_type_b]

    fill_function = random.choice(fill_functions)
    return fill_function(chart_json=chart_json)

In [96]:
messages=[]

In [97]:
for chart_json in final_dataset:
    messages.append([
        {"role": "user", "content": f"Paraphrase the following prompt: '{fill_template(chart_json=chart_json)}'"}
    ])

In [98]:
len(messages)

200

In [99]:
messages[:10]

[[{'role': 'user',
   'content': "Paraphrase the following prompt: 'Please give me a line chart in ChartsJS for highest points of acrs.us stock with the labels 2015-10-30, 2016-04-29, 2016-06-24, 2016-09-20, 2016-09-22, 2016-10-10, 2016-10-18, 2016-11-01, 2016-11-09, 2016-11-16, 2017-01-11, 2017-02-15, 2017-03-20, 2017-03-22, 2017-06-09, 2017-06-23 and their respective values 14.85, 19.89, 18.98, 24.74, 25.075, 25.3, 22.8, 21.23, 22.73, 24.93, 29.24, 28.25, 28.98, 29.65, 25.52, 27.19.'"}],
 [{'role': 'user',
   'content': "Paraphrase the following prompt: 'Develop a line chart in ChartsJS illustrating highest points of acrs.us stock with data points: 2015-12-28 (28.97), 2016-02-01 (22.26), 2016-03-15 (16.83), 2016-03-31 (19.24), 2016-04-12 (20.52), 2016-05-19 (20.6), 2016-10-18 (22.8), 2016-10-20 (22.4), 2016-11-17 (23.96), 2017-01-18 (28.5), 2017-01-20 (27.88), 2017-02-15 (28.25), 2017-06-07 (24.9), 2017-09-05 (26.38), 2017-10-16 (26.62), 2017-10-26 (25.29)'"}],
 [{'role': 'user',
   

In [100]:
final_prompts = []

In [101]:
for i, message in enumerate(messages):
    output = pipe(message, **generation_args)
    final_prompts.append(output[0]['generated_text'])
    print(output[0]['generated_text'])

generated 0
 Craft an XY line chart using ChartsJS that illustrates the peak values of ACRS.US stock. The chart should feature the dates: October 30, 2015; April 29, 2016; June 24, 2016; September 20, 2016; September 22, 2016; October 10, 2016; October 18, 2016; November 1, 2016; November 9, 2016; November 16, 2016; January 11, 2017; February 15, 2017; March 20, 2017; March 22, 2017; June 9, 2017; and June 23, 2017. Correspond these dates with their respective stock values: 14.85, 19.89, 18.98, 24.74, 25.075, 25.3, 22.8, 21.23, 22.73, 24.93, 29.24, 28.25, 28.98, 29.65, 25.52, and 27.19.
generated 20
 Create a line chart using ChartsJS that illustrates the peak values of adm.us stock, based on the provided data points: June 11, 2014 (19.93), October 10, 2014 (14.75), October 16, 2014 (14.57), June 24, 2015 (27.5), August 27, 2015 (20.36), October 2, 2015 (17.16), January 28, 2016 (18.8), February 12, 2016 (14.37), March 28, 2016 (14.595), December 7, 2016 (15.72), December 12, 2016 (15.

In [102]:
with open("final_prompts.json", "w") as f:
    json.dump(final_prompts, f, indent=4, ensure_ascii=False)